In [1]:
!nvidia-smi

Sun May 26 08:29:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Импорт библиотек

In [2]:
from google.colab import drive
import torch
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import DefaultDataCollator
from transformers import TrainingArguments, Trainer

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

## Подключение Google Drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


# Подготовка к тонкой настройке

## Инициализация модели

In [7]:
model_name = 'ai-forever/ruBert-base'

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [9]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(DEVICE)

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

## Загрузка набора данных SberQuAD

In [10]:
sberquad = load_dataset('sberquad')
sberquad

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [14]:
# Функция для обработки набора данных
def preprocess_function(examples):
    # Тексты вопросов
    questions = [q.strip() for q in examples['question']]
    # Тексты контекстов
    contexts = [c.strip() for c in examples['context']]

    # Входы для модели
    inputs = tokenizer(
        questions,
        contexts,
        max_length=256,
        truncation='only_second',
        padding='max_length',
        return_offsets_mapping=True,
        return_tensors='pt',
    )

    offset_mapping = inputs.pop('offset_mapping')
    answers = examples['answers']
    start_positions = []
    end_positions = []

    # Поиск стартовых и конечных позиций ответов
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer['answer_start'][0]
        end_char = answer['answer_start'][0] + len(answer['text'][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Если ответ не полностью внутри контекста, отметить начало и конец за (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions

    return inputs

### Проверка обработки набора данных

In [15]:
test = preprocess_function(sberquad['train'][:100])

In [16]:
tokenizer.decode(test['input_ids'][0])

'[CLS] чем представлены органические остатки? [SEP] в протерозоиских отложениях органические остатки встречаются намного чаще, чем в археиских. они представлены известковыми выделениями сине - зеленых водорослеи, ходами червеи, остатками кишечнополостных. кроме известковых водорослеи, к числу древнеиших растительных остатков относятся скопления графито - углистого вещества, образовавшегося в результате разложения corycium enigmaticum. в кремнистых сланцах железоруднои формации канады наидены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. в железистых кварцитах севернои америки и сибири обнаружены железистые продукты жизнедеятельности бактерии. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

### Обработка всего набора данных

In [18]:
%%time

tokenized_sberquad = sberquad.map(preprocess_function,
                                  batched=True,
                                  remove_columns=sberquad['train'].column_names)

Map:   0%|          | 0/45328 [00:00<?, ? examples/s]

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

Map:   0%|          | 0/23936 [00:00<?, ? examples/s]

CPU times: user 3min 1s, sys: 845 ms, total: 3min 2s
Wall time: 2min 41s


# Тонкая настройка модели

In [19]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/ruBERT_QA',
    save_strategy='no',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sberquad['train'],
    eval_dataset=tokenized_sberquad['validation'],
    tokenizer=tokenizer,
    data_collator=DefaultDataCollator(),
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,1.192400,1.732502


TrainOutput(global_step=2833, training_loss=0.8962823320436596, metrics={'train_runtime': 2338.0932, 'train_samples_per_second': 19.387, 'train_steps_per_second': 1.212, 'total_flos': 5922029694664704.0, 'train_loss': 0.8962823320436596, 'epoch': 1.0})

In [26]:
trainer.save_model('/content/drive/MyDrive/ruBERT_QA_1_epoch')

# Проверка модели

In [27]:
question = 'Что я планирую?'
context = 'Я был рожден не каким-то там человеком, а ящером, и теперь я готов захватить мир'

In [28]:
from transformers import pipeline

question_answerer = pipeline('question-answering', model=model, tokenizer=tokenizer)
result = question_answerer(question=question, context=context)

print(f'Вопрос: {question}')
print(result)

Вопрос: Кто я?
{'score': 0.09262501448392868, 'start': 61, 'end': 65, 'answer': 'Иван'}
